# Chapter 18 - Request-EndPoint-Response (REPR)

- Pattern added on top of Vertical Slice and CQS patterns
- Better for building rest apis than MVC
- MVC framework could work to implement this pattern but Minimal API fits better

## Three components

- Request: Input DTO (Can be treated as a Command in CQS)
- EndPoint: Handler with business logic
- Response: Output DTO

## Project: SimpleEndpoints

In [1]:
string[] args = {"--urls","http://localhost:7000"};

### Feature: ShuffleText

In [4]:
public class ShuffleText
{
    public record class Request(string Text);
    public record class Response(string Text);
    public class Endpoint
    {
        public Response Handle(Request request)
        {
            var chars = request.Text.ToArray();
            Random.Shared.Shuffle(chars);
            return new Response(new string(chars));
        }
    }
}

### Feature: RandomNumber

In [5]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
using Microsoft.AspNetCore.Http;

public class RandomNumber
{
    public record class Request(int Amount, int Min, int Max);
    public record class Response(IEnumerable<int> Numbers);
    public class Handler
    {
        public Response Handle(Request request)
        {
            var result = new int[request.Amount];
            for (var i = 0; i < request.Amount; i++)
            {
                result[i] = Random.Shared.Next(request.Min, request.Max);
            }
            return new Response(result);
        }
    }

    public static Response Endpoint([AsParameters] Request query, Handler handler)
        => handler.Handle(query);
}


### Feature: UpperCase

In [14]:
// Extension methods must be at the top level in .NET interactive, but would otherwise be in its own class
// public static class UpperCase
// {
    public record class Request(string Text);
    public record class Response(string Text);
    public class Handler
    {
        public Response Handle(Request request)
        {
            return new Response(request.Text.ToUpper());
        }
    }

    public static IServiceCollection AddUpperCase(this IServiceCollection services)
    {
        return services.AddSingleton<Handler>();
    }

    public static IEndpointRouteBuilder MapUpperCase(this IEndpointRouteBuilder endpoints)
    {
        endpoints.MapGet(
            "/upper-case/{Text}",
            ([AsParameters] Request query, Handler handler)
                => handler.Handle(query)
        );
        return endpoints;
    }
// }

In [16]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.DependencyInjection.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.DependencyInjection.Abstractions.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Logging.Abstractions.dll"


using Microsoft.AspNetCore.Http;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.DependencyInjection.Extensions;

var builder = WebApplication.CreateBuilder(args);
builder.Services.AddSingleton<ShuffleText.Endpoint>();
builder.Services.AddSingleton<RandomNumber.Handler>();
builder.Services.AddUpperCase();


var app = builder.Build();

app.MapGet("/shuffle-text/{text}", ([AsParameters] ShuffleText.Request query, ShuffleText.Endpoint endpoint)
     => endpoint.Handle(query));

app.MapGet("/random-number/{Amount}/{Min}/{Max}", RandomNumber.Endpoint);

app.MapUpperCase();

app.RunAsync();


info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:7000
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\jason\training\dotnet\Architecting-ASP.NET-Core-Applications-3E\MyNotes


In [7]:
using System.Net.Http;

var httpClient = new HttpClient();
var response = await httpClient.GetAsync("http://localhost:7000/shuffle-text/I%20love%20ASP.NET%20Core");
//response.EnsureSuccessStatusCode();
display(response);
var responseData = await response.Content.ReadAsStringAsync();
responseData.DisplayAs("application/json")

StatusCode: 200, ReasonPhrase: 'OK', Version: 1.1, Content: System.Net.Http.HttpConnectionResponseContent, Headers:\r\n{\r\n Date: Thu, 14 Aug 2025 15:11:45 GMT\r\n Server: Kestrel\r\n Transfer-Encoding: chunked\r\n Content-Type: application/json; charset=utf-8\r\n} Version 1.1 Major 1 Minor 1 Build -1 Revision -1 MajorRevision -1 MinorRevision -1 Content System.Net.Http.HttpConnectionResponseContent Headers [ [Content-Type, System.String[]] ] Allow [ ] Count 0 IsReadOnly False (values) [ ] ContentDisposition <null> ContentEncoding [ ] Count 0 IsReadOnly False (values) [ ] ContentLanguage [ ] Count 0 IsReadOnly False (values) [ ] ContentLength 30 ContentLocation <null> ContentMD5 <null> ContentRange <null> ContentType application/json; charset=utf-8 CharSet utf-8 Parameters [ charset=utf-8 ] Count 1 IsReadOnly False (values) index value 0 charset=utf-8 Name charset Value utf-8 MediaType application/json Expires <null> LastModified <null> NonValidated [ [Content-Type, application/json; charset=utf-8], [Content-Length, 30] ] Count 2 (values) index value 0 [Content-Type, application/json; charset=utf-8] Key Content-Type Value [ application/json; charset=utf-8 ] Count 1 (values) [ application/json; charset=utf-8 ] 1 [Content-Length, 30] Key Content-Length Value [ 30 ] Count 1 (values) [ 30 ] (values) index value 0 [Content-Type, System.String[]] Key Content-Type Value [ application/json; charset=utf-8 ] 1 [Content-Length, System.String[]] Key Content-Length Value [ 30 ] StatusCode OK ReasonPhrase OK Headers [ [Date, System.String[]], [Server, System.String[]], [Transfer-Encoding, System.String[]] ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ Kestrel ] Count 1 IsReadOnly False (values) index value 0 Kestrel Product Kestrel Name Kestrel Version <null> Comment <null> Vary [ ] Count 0 IsReadOnly False (values) [ ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ ] Count 0 IsReadOnly False (values) [ ] ConnectionClose <null> Date 2025-08-14 15:11:45Z Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ chunked ] Count 1 IsReadOnly False (values) index value 0 chunked Value chunked Parameters [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked True Upgrade [ ] Count 0 IsReadOnly False (values) (empty) Via [ ] Count 0 IsReadOnly False (values) (empty) Warning [ ] Count 0 IsReadOnly False (values) (empty) NonValidated [ [Date, Thu, 14 Aug 2025 15:11:45 GMT], [Server, Kestrel], [Transfer-Encoding, chunked] ] Count 3 (values) index value 0 [Date, Thu, 14 Aug 2025 15:11:45 GMT] Key Date Value [ Thu, 14 Aug 2025 15:11:45 GMT ] Count 1 (values) [ Thu, 14 Aug 2025 15:11:45 GMT ] 1 [Server, Kestrel] Key Server Value [ Kestrel ] Count 1 (values) [ Kestrel ] 2 [Transfer-Encoding, chunked] Key Transfer-Encoding Value [ chunked ] Count 1 (values) [ chunked ] (values) index value 0 [Date, System.String[]] Key Date Value [ Thu, 14 Aug 2025 15:11:45 GMT ] 1 [Server, System.String[]] Key Server Value [ Kestrel ] 2 [Transfer-Encoding, System.String[]] Key Transfer-Encoding Value [ chunked ] TrailingHeaders [ ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ ] Count 0 IsReadOnly False (values) (empty) Vary [ ] Count 0 IsReadOnly False (values) [ ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ ] Count 0 IsReadOnly False (values) [ ] ConnectionClose <null> Date <null> Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked <null> Upgrade [ ] Count 0 IsReadOnly False (values)

In [8]:
using System.Net.Http;

var httpClient = new HttpClient();
var response = await httpClient.GetAsync("http://localhost:7000/random-number/5/0/100");
//response.EnsureSuccessStatusCode();
display(response);
var responseData = await response.Content.ReadAsStringAsync();
responseData.DisplayAs("application/json")

StatusCode: 200, ReasonPhrase: 'OK', Version: 1.1, Content: System.Net.Http.HttpConnectionResponseContent, Headers:\r\n{\r\n Date: Thu, 14 Aug 2025 15:12:32 GMT\r\n Server: Kestrel\r\n Transfer-Encoding: chunked\r\n Content-Type: application/json; charset=utf-8\r\n} Version 1.1 Major 1 Minor 1 Build -1 Revision -1 MajorRevision -1 MinorRevision -1 Content System.Net.Http.HttpConnectionResponseContent Headers [ [Content-Type, System.String[]] ] Allow [ ] Count 0 IsReadOnly False (values) [ ] ContentDisposition <null> ContentEncoding [ ] Count 0 IsReadOnly False (values) [ ] ContentLanguage [ ] Count 0 IsReadOnly False (values) [ ] ContentLength 28 ContentLocation <null> ContentMD5 <null> ContentRange <null> ContentType application/json; charset=utf-8 CharSet utf-8 Parameters [ charset=utf-8 ] Count 1 IsReadOnly False (values) index value 0 charset=utf-8 Name charset Value utf-8 MediaType application/json Expires <null> LastModified <null> NonValidated [ [Content-Type, application/json; charset=utf-8], [Content-Length, 28] ] Count 2 (values) index value 0 [Content-Type, application/json; charset=utf-8] Key Content-Type Value [ application/json; charset=utf-8 ] Count 1 (values) [ application/json; charset=utf-8 ] 1 [Content-Length, 28] Key Content-Length Value [ 28 ] Count 1 (values) [ 28 ] (values) index value 0 [Content-Type, System.String[]] Key Content-Type Value [ application/json; charset=utf-8 ] 1 [Content-Length, System.String[]] Key Content-Length Value [ 28 ] StatusCode OK ReasonPhrase OK Headers [ [Date, System.String[]], [Server, System.String[]], [Transfer-Encoding, System.String[]] ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ Kestrel ] Count 1 IsReadOnly False (values) index value 0 Kestrel Product Kestrel Name Kestrel Version <null> Comment <null> Vary [ ] Count 0 IsReadOnly False (values) [ ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ ] Count 0 IsReadOnly False (values) [ ] ConnectionClose <null> Date 2025-08-14 15:12:32Z Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ chunked ] Count 1 IsReadOnly False (values) index value 0 chunked Value chunked Parameters [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked True Upgrade [ ] Count 0 IsReadOnly False (values) (empty) Via [ ] Count 0 IsReadOnly False (values) (empty) Warning [ ] Count 0 IsReadOnly False (values) (empty) NonValidated [ [Date, Thu, 14 Aug 2025 15:12:32 GMT], [Server, Kestrel], [Transfer-Encoding, chunked] ] Count 3 (values) index value 0 [Date, Thu, 14 Aug 2025 15:12:32 GMT] Key Date Value [ Thu, 14 Aug 2025 15:12:32 GMT ] Count 1 (values) [ Thu, 14 Aug 2025 15:12:32 GMT ] 1 [Server, Kestrel] Key Server Value [ Kestrel ] Count 1 (values) [ Kestrel ] 2 [Transfer-Encoding, chunked] Key Transfer-Encoding Value [ chunked ] Count 1 (values) [ chunked ] (values) index value 0 [Date, System.String[]] Key Date Value [ Thu, 14 Aug 2025 15:12:32 GMT ] 1 [Server, System.String[]] Key Server Value [ Kestrel ] 2 [Transfer-Encoding, System.String[]] Key Transfer-Encoding Value [ chunked ] TrailingHeaders [ ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ ] Count 0 IsReadOnly False (values) (empty) Vary [ ] Count 0 IsReadOnly False (values) [ ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ ] Count 0 IsReadOnly False (values) [ ] ConnectionClose <null> Date <null> Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked <null> Upgrade [ ] Count 0 IsReadOnly False (values)

In [17]:
using System.Net.Http;

var httpClient = new HttpClient();
var response = await httpClient.GetAsync("http://localhost:7000/upper-case/I%20love%20ASP.NET%20Core");
//response.EnsureSuccessStatusCode();
display(response);
var responseData = await response.Content.ReadAsStringAsync();
responseData.DisplayAs("application/json")

StatusCode: 200, ReasonPhrase: 'OK', Version: 1.1, Content: System.Net.Http.HttpConnectionResponseContent, Headers:\r\n{\r\n Date: Thu, 14 Aug 2025 15:24:02 GMT\r\n Server: Kestrel\r\n Transfer-Encoding: chunked\r\n Content-Type: application/json; charset=utf-8\r\n} Version 1.1 Major 1 Minor 1 Build -1 Revision -1 MajorRevision -1 MinorRevision -1 Content System.Net.Http.HttpConnectionResponseContent Headers [ [Content-Type, System.String[]] ] Allow [ ] Count 0 IsReadOnly False (values) [ ] ContentDisposition <null> ContentEncoding [ ] Count 0 IsReadOnly False (values) [ ] ContentLanguage [ ] Count 0 IsReadOnly False (values) [ ] ContentLength 30 ContentLocation <null> ContentMD5 <null> ContentRange <null> ContentType application/json; charset=utf-8 CharSet utf-8 Parameters [ charset=utf-8 ] Count 1 IsReadOnly False (values) index value 0 charset=utf-8 Name charset Value utf-8 MediaType application/json Expires <null> LastModified <null> NonValidated [ [Content-Type, application/json; charset=utf-8], [Content-Length, 30] ] Count 2 (values) index value 0 [Content-Type, application/json; charset=utf-8] Key Content-Type Value [ application/json; charset=utf-8 ] Count 1 (values) [ application/json; charset=utf-8 ] 1 [Content-Length, 30] Key Content-Length Value [ 30 ] Count 1 (values) [ 30 ] (values) index value 0 [Content-Type, System.String[]] Key Content-Type Value [ application/json; charset=utf-8 ] 1 [Content-Length, System.String[]] Key Content-Length Value [ 30 ] StatusCode OK ReasonPhrase OK Headers [ [Date, System.String[]], [Server, System.String[]], [Transfer-Encoding, System.String[]] ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ Kestrel ] Count 1 IsReadOnly False (values) index value 0 Kestrel Product Kestrel Name Kestrel Version <null> Comment <null> Vary [ ] Count 0 IsReadOnly False (values) [ ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ ] Count 0 IsReadOnly False (values) [ ] ConnectionClose <null> Date 2025-08-14 15:24:02Z Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ chunked ] Count 1 IsReadOnly False (values) index value 0 chunked Value chunked Parameters [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked True Upgrade [ ] Count 0 IsReadOnly False (values) (empty) Via [ ] Count 0 IsReadOnly False (values) (empty) Warning [ ] Count 0 IsReadOnly False (values) (empty) NonValidated [ [Date, Thu, 14 Aug 2025 15:24:02 GMT], [Server, Kestrel], [Transfer-Encoding, chunked] ] Count 3 (values) index value 0 [Date, Thu, 14 Aug 2025 15:24:02 GMT] Key Date Value [ Thu, 14 Aug 2025 15:24:02 GMT ] Count 1 (values) [ Thu, 14 Aug 2025 15:24:02 GMT ] 1 [Server, Kestrel] Key Server Value [ Kestrel ] Count 1 (values) [ Kestrel ] 2 [Transfer-Encoding, chunked] Key Transfer-Encoding Value [ chunked ] Count 1 (values) [ chunked ] (values) index value 0 [Date, System.String[]] Key Date Value [ Thu, 14 Aug 2025 15:24:02 GMT ] 1 [Server, System.String[]] Key Server Value [ Kestrel ] 2 [Transfer-Encoding, System.String[]] Key Transfer-Encoding Value [ chunked ] TrailingHeaders [ ] AcceptRanges [ ] Count 0 IsReadOnly False (values) [ ] Age <null> ETag <null> Location <null> ProxyAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) RetryAfter <null> Server [ ] Count 0 IsReadOnly False (values) (empty) Vary [ ] Count 0 IsReadOnly False (values) [ ] WwwAuthenticate [ ] Count 0 IsReadOnly False (values) (empty) CacheControl <null> Connection [ ] Count 0 IsReadOnly False (values) [ ] ConnectionClose <null> Date <null> Pragma [ ] Count 0 IsReadOnly False (values) (empty) Trailer [ ] Count 0 IsReadOnly False (values) [ ] TransferEncoding [ ] Count 0 IsReadOnly False (values) (empty) TransferEncodingChunked <null> Upgrade [ ] Count 0 IsReadOnly False (values)

In [18]:
await app.StopAsync();

## Project: E-commerce app

[App Src Code](../C18/REPR/Web/Program.cs)

### Features

* Product catalog
* Shopping cart

The next 2 chapters will build upon this.

### Stack

- Minimal API as backbone
- FluentValidation
- Mapperly for mapping dto to model and vice versa
- ExceptionMapper - Helps handle exceptions globally
- EF Core


```bash
dotnet add package FluentValidation.AspNetCore
dotnet add package ForEvolve.ExceptionMapper
dotnet add package ForEvolve.FluentValidation.AspNetCore.Http
dotnet add package Microsoft.EntityFrameworkCore.InMemory
dotnet add package Riok.Mapperly
```

### File org

Features > {Areas i.e. `Baskets`} > {Features i.e. `AddItem.cs`}

Extension methods to map features are in `Features/Features.cs`

## Exception middleware

- Can use exceptions to propogate errors to api clients

```csharp
public class MyExceptionMiddleware : IMiddleware
{
    public async Task InvokeAsync(HttpContext context, RequestDelegate next)
    {
        var exceptionHandlerPathFeature = context.Features
            .Get<IExceptionHandlerFeature>() ?? throw new NotSupportedException();

        var exception = exceptionHandlerPathFeature.Error;
        await context.Response.WriteAsJsonAsync(new
        {
            Error = exception.Message
        });
        await next(context);
    }
}
```

Register the middleware

```csharp
builder.Services.AddSingleton<MyExceptionMiddleware>();

app.UseExceptionHandler(errorApp =>
{
    errorApp.UseMiddleware<MyExceptionMiddleware>();
});
```

### ForEvolve.ExceptionMapper

- Allows us to map exceptions to status codes
- Outof the box `ProblemDetails` object
    * Debug info included in dev


## Gray box testing (Integration tests)

See [C18\REPR\Web.Tests\WebApplication.cs](../C18/REPR/Web.Tests/WebApplication.cs) for examples of gray box testing in ASP.NET Core applications.